In [106]:
# epsilon 값 설정
epsilon = 0.05

# 모델 학습
model.fit(X_train, y_train_cat, epochs=50, batch_size=16, verbose=0)

# 비일치성 측도 계산 및 예측 신뢰 구간 설정
y_pred_test = model.predict(X_test)
prediction_intervals = []
actual_labels = []

for i in range(len(X_test)):
    pred = y_pred_test[i]
    true_label = y_test[i]
    actual_labels.append(true_label)
    
    class_probabilities = pred
    sorted_indices = np.argsort(class_probabilities)[::-1]  # 확률이 높은 순서대로 정렬
    cumulative_prob = 0.0
    prediction_interval = []
    
    for idx in sorted_indices:
        cumulative_prob += class_probabilities[idx]
        prediction_interval.append(idx)
        if cumulative_prob >= 1.0 - epsilon:
            break
    
    prediction_intervals.append(prediction_interval)

# 결과 출력
for i, interval in enumerate(prediction_intervals):
    print(f"Sample {i}: Predicted classes: {interval}, Actual label: {actual_labels[i]}")


1/1 [==============================] - 0s 63ms/step
Sample 0: Predicted classes: [1], Actual label: 1
Sample 1: Predicted classes: [0], Actual label: 0
Sample 2: Predicted classes: [2], Actual label: 2
Sample 3: Predicted classes: [1, 2], Actual label: 1
Sample 4: Predicted classes: [1], Actual label: 1
Sample 5: Predicted classes: [0], Actual label: 0
Sample 6: Predicted classes: [1], Actual label: 1
Sample 7: Predicted classes: [2], Actual label: 2
Sample 8: Predicted classes: [1, 2], Actual label: 1
Sample 9: Predicted classes: [1], Actual label: 1
Sample 10: Predicted classes: [2, 1], Actual label: 2
Sample 11: Predicted classes: [0], Actual label: 0
Sample 12: Predicted classes: [0], Actual label: 0
Sample 13: Predicted classes: [0], Actual label: 0
Sample 14: Predicted classes: [0], Actual label: 0
Sample 15: Predicted classes: [1], Actual label: 1
Sample 16: Predicted classes: [2], Actual label: 2
Sample 17: Predicted classes: [1], Actual label: 1
Sample 18: Predicted classes: [

# Tutorial on Coformal Prediction

위 논문의 예제와 알고리즘을 파이썬으로 구현한 코드입니다.

## 3.3 The Betting Interpretation of Exchangeability

In [128]:
import numpy as np

class BackwardLookingBettingGame:
    def __init__(self, N):
        self.N = N
        self.KN = 1.0  # Initial capital
        self.bag = np.random.randint(0, 5, N)  # Assume z can be 0 to 4 for simplicity
        self.current_index = N - 1
    
    def bet(self):
        while self.current_index >= 0:
            current_value = self.bag[self.current_index]
            current_bag = self.bag[:self.current_index + 1]
            possible_values, counts = np.unique(current_bag, return_counts=True)
            bets = {}
            
            for value, count in zip(possible_values, counts):
                k = count
                n = len(current_bag)
                odds_for = k / n
                odds_against = 1 - odds_for
                bet_amount = min(self.KN, 0.1)  # Bill bets up to 0.1 units or remaining capital
                
                if current_value == value:
                    win_amount = bet_amount * odds_for / odds_against
                    bets[value] = win_amount
                    self.KN += win_amount
                else:
                    lose_amount = bet_amount * odds_against / odds_for
                    bets[value] = -lose_amount
                    self.KN -= lose_amount
            
            print(f"Step {self.current_index}: Bets {bets}, Current Capital: {self.KN}")
            
            if self.KN <= 0:
                print("Bill has run out of capital!")
                break
            
            self.current_index -= 1
    
    def announce_result(self):
        print("Joe announces the bag B:", self.bag)
        self.bet()
        print("Final Capital:", self.KN)

# 게임 초기화
N = 20
game = BackwardLookingBettingGame(N)
game.announce_result()


Joe announces the bag B: [4 1 3 1 0 3 3 0 0 1 2 4 3 1 4 0 0 4 2 3]
Step 19: Bets {0: -0.30000000000000004, 1: -0.4000000000000001, 2: -0.9, 3: -0.20000000000000004, 4: 3.2000000000000006}, Current Capital: 2.4000000000000004
Step 18: Bets {0: -0.28, 1: -0.37500000000000006, 2: 0.011764705882352941, 3: -0.37500000000000006, 4: -0.37500000000000006}, Current Capital: 1.006764705882353
Step 17: Bets {0: -0.26, 1: -0.35000000000000003, 2: -1.7, 3: 4.561323529411765, 4: 0.02857142857142857}, Current Capital: 3.2866596638655463
Step 16: Bets {0: 0.04166666666666668, 1: -0.325, 2: -1.6, 3: -0.325, 4: -0.4666666666666666}, Current Capital: 0.611659663865546
Step 15: Bets {0: 0.03333333333333333, 1: -0.30000000000000004, 2: -1.5, 3: 3.4650210084033617, 4: -0.43333333333333335}, Current Capital: 1.8766806722689078
Step 14: Bets {0: -0.4000000000000001, 1: -0.2750000000000001, 2: -1.4000000000000001, 3: 0.5453781512605047, 4: 0.025000000000000005}, Current Capital: 0.3720588235294121
Step 13: Bet

## 4.2 Conformal Prediction from Old Examples Alone

In [173]:
def conformal_algorithm(A, epsilon, examples, z):
    n = len(examples)
    provisional_set = examples + [z]
    alpha_values = []

    # Step 1: Provisionally set zn := z
    zn = z

    # Step 2: Calculate α for each example in the provisional set
    for i in range(n + 1):
        z_i = provisional_set[i]
        
        # Construct the bag without z_i
        z_star = provisional_set[:i] + provisional_set[i+1:]
        
        # Calculate α using the nonconformity measure A
        alpha = A(z_star, z_i)
        alpha_values.append(alpha)
        
    # Calculate α_n
    alpha_n = A(provisional_set[:n], zn)
    
    # Step 3: Calculate p_z
    sorted_alphas = np.sort(alpha_values)
    rank_of_alpha = np.sum(sorted_alphas >= alpha_n)

    pz = rank_of_alpha / n

    # Step 4: Decide whether to include z in γ
    include_z = (pz > epsilon)
    
    return include_z

# Example nonconformity measure: absolute difference
def nonconformity_measure(z_star, z):
    return np.abs(np.array(z_star) - z)

In [174]:
# Example data
examples = [1, 2, 3, 4]
z = 5
epsilon = 0.5

# Run the algorithm
include_z = conformal_algorithm(nonconformity_measure, epsilon, examples, z)
print(f"Include z={z} in γε(z1,...,zn−1): {include_z}")


Include z=5 in γε(z1,...,zn−1): True


근데 어떻게 설정해도 다 True가 나오는 이슈......
왜일까..??

## 4.3 Conformal Prediction Using a New Object

### Algorithm

In [175]:
def conformal_algorithm(A, epsilon, examples, xn, y):
    n = len(examples)
    provisional_set = examples + [(xn, y)]
    alpha_values = []

    # Step 1: Provisionally set zn := (xn, y)
    zn = (xn, y)

    # Step 2: Calculate α for each example in the provisional set
    for i in range(n + 1):
        z_i = provisional_set[i]
        
        # Construct the bag without z_i
        z_star = provisional_set[:i] + provisional_set[i+1:]
        
        # Calculate α using the nonconformity measure A
        alpha = A(z_star, z_i)
        alpha_values.append(alpha)
        
    # Calculate α_n
    alpha_n = A([example[0] for example in provisional_set[:n]], zn[0])

    # Step 3: Calculate p_y
    sorted_alphas = np.sort(alpha_values)
    rank_of_alpha = np.sum(sorted_alphas >= alpha_n)

    py = rank_of_alpha/n

    # Step 4: Decide whether to include y in Γ
    include_y = (py > epsilon)
    
    return include_y

# Example nonconformity measure: absolute difference for xn as a vector and y as a scalar
def nonconformity_measure(z_star, z):
    return np.sum(np.abs(z_star) - z)

In [176]:
# Example data 1
examples = [(1, 0), (2, 0), (3, 1), (4, 1)]  # examples: (xn, y)
xn = 0
y = 1
epsilon = 0.1

# Run the algorithm
include_y = conformal_algorithm(nonconformity_measure, epsilon, examples, xn, y)
print(f"Include y={y} in Γε(z1,...,zn−1,xn): {include_y}")

Include y=1 in Γε(z1,...,zn−1,xn): False


In [177]:
# Example data 2
examples = [(1, 0), (2, 0), (3, 1), (4, 1)]  # examples: (xn, y)
xn = 1
y = 1
epsilon = 0.1

# Run the algorithm
include_y = conformal_algorithm(nonconformity_measure, epsilon, examples, xn, y)
print(f"Include y={y} in Γε(z1,...,zn−1,xn): {include_y}")

Include y=1 in Γε(z1,...,zn−1,xn): True


### 4.3.1. Example : Classifying IRIS Flowers

- sklearn의 iris dataset을 이용했습니다.
- Neural Network Model
- X : sepal_length, sepal_width, petal_length, petal_width
- y=0 'setosa'
- y=1 'versicolor'
- y=2 'virginica

#### (1) NN

In [204]:
# 데이터 로드 및 전처리
data = load_iris()
X = data.data 
y = data.target

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encoding
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Modeling
model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    #Dense(64, activation='relu'),
    Dense(3), #3 labels
    Softmax()
])

# Model Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [205]:
# Conformal Prediction을 위한 비일치성 측도 계산 함수
def calculate_nonconformity(scores, true_label):
    return 1.0 - scores[true_label]

In [206]:
# epsilon 값 설정
epsilon = 0.05

# 모델 학습
model.fit(X_train, y_train_cat, epochs=50, batch_size=16, verbose=0)

# 비일치성 측도 계산 및 예측 신뢰 구간 설정
y_pred_test = model.predict(X_test)
prediction_intervals = []
actual_labels = []

for i in range(len(X_test)):
    pred = y_pred_test[i]
    true_label = y_test[i]
    actual_labels.append(true_label)
    
    class_probabilities = pred
    sorted_indices = np.argsort(class_probabilities)[::-1]  # 확률이 높은 순서대로 정렬
    cumulative_prob = 0.0
    prediction_interval = []
    
    for idx in sorted_indices:
        cumulative_prob += class_probabilities[idx]
        prediction_interval.append(idx)
        if cumulative_prob >= 1.0 - epsilon:
            break
    
    prediction_intervals.append(prediction_interval)

# 결과 출력
for i, interval in enumerate(prediction_intervals):
    print(f"Sample {i}: Predicted classes: {interval}, Actual label: {actual_labels[i]}")


1/1 [==============================] - 0s 55ms/step
Sample 0: Predicted classes: [1, 2], Actual label: 1
Sample 1: Predicted classes: [0], Actual label: 0
Sample 2: Predicted classes: [2], Actual label: 2
Sample 3: Predicted classes: [1, 2], Actual label: 1
Sample 4: Predicted classes: [1, 2], Actual label: 1
Sample 5: Predicted classes: [0], Actual label: 0
Sample 6: Predicted classes: [1, 2, 0], Actual label: 1
Sample 7: Predicted classes: [2, 1], Actual label: 2
Sample 8: Predicted classes: [1, 2], Actual label: 1
Sample 9: Predicted classes: [1, 2], Actual label: 1
Sample 10: Predicted classes: [2, 1], Actual label: 2
Sample 11: Predicted classes: [0], Actual label: 0
Sample 12: Predicted classes: [0], Actual label: 0
Sample 13: Predicted classes: [0], Actual label: 0
Sample 14: Predicted classes: [0], Actual label: 0
Sample 15: Predicted classes: [1, 2, 0], Actual label: 1
Sample 16: Predicted classes: [2], Actual label: 2
Sample 17: Predicted classes: [1, 2], Actual label: 1
Samp

In [191]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_cat)
print(f'Test accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 112ms/step - loss: 0.1294 - accuracy: 0.9667
Test accuracy: 0.9667


#### (2) Distance to the nearest neighbor


In [213]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.neighbors import NearestNeighbors

# Load IRIS dataset
iris = load_iris()
X = iris.data
y = iris.target

# Select samples for 'setosa' (y=0) and 'versicolor' (y=1)
X_setosa = X[y == 0]
X_versicolor = X[y == 1]

# Calculate nonconformity scores using nearest neighbor distance
def nonconformity_measure(sample, X_class):
    
    # Compute distances from 'sample' to all samples in 'X_class'
    distances = np.linalg.norm(X_class - sample, axis=1)
    
    # Exclude distance to the sample itself (where distance is zero)
    nonzero_distances = distances[distances != 0]
    
    # Find the minimum distance
    min_distance = np.min(nonzero_distances) if len(nonzero_distances) > 0 else 0
    return min_distance

# Compute nonconformity scores for 'setosa' and 'versicolor' classes
def compute_nonconformity_scores(X, X_setosa, X_versicolor):
    n = len(X)
    alpha_setosa = np.zeros(n)
    alpha_versicolor = np.zeros(n)
    
    for i in range(n):
        sample = X[i]
        alpha_setosa[i] = nonconformity_measure(sample, X_setosa)
        alpha_versicolor[i] = nonconformity_measure(sample, X_versicolor)
    
    return alpha_setosa, alpha_versicolor

# Compute nonconformity scores for all samples
alpha_setosa, alpha_versicolor = compute_nonconformity_scores(X, X_setosa, X_versicolor)

# Print nonconformity scores for first 10 samples as an example
for i in range(10):
    print(f"Sample {i}: Setosa score = {round(alpha_setosa[i],4)}, Versicolor score = {round(alpha_versicolor[i],4)}")


Sample 0: Setosa score = 0.1, Versicolor score = 2.0905
Sample 1: Setosa score = 0.1414, Versicolor score = 1.9131
Sample 2: Setosa score = 0.1414, Versicolor score = 2.0857
Sample 3: Setosa score = 0.1414, Versicolor score = 1.9157
Sample 4: Setosa score = 0.1414, Versicolor score = 2.1424
Sample 5: Setosa score = 0.3317, Versicolor score = 2.0567
Sample 6: Setosa score = 0.2236, Versicolor score = 2.064
Sample 7: Setosa score = 0.1, Versicolor score = 1.9698
Sample 8: Setosa score = 0.1414, Versicolor score = 2.005
Sample 9: Setosa score = 0.1, Versicolor score = 1.9105
